# What does this script do?
This script runs through a list of study ids, pulls down the imaging table produced in iBroker, concatenates the tables together, and saves them to an output csv file.

# Installation
To install the python requirements, run
```pip install -r requirements.txt```

Selenium additionally requires the installation of a driver for your browser (Chrome in this example). See here for details: https://selenium-python.readthedocs.io/installation.html

In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from tqdm.auto import tqdm

In [ ]:
# insert real values here; study id file should be a text file with a list of study ids, one per line
username = "foo"
password = "bar"
study_id_file = "16352A_study_ids.csv"

In [2]:
study_ids = pd.read_csv(study_id_file, header=None, names=["id"])["id"].tolist()

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("headless")
driver = webdriver.Chrome(options=options)
driver.get("http://cw2radiis03.uchad.uchospitals.edu/ibroker/")

In [ ]:
elem = driver.find_element_by_name("tbxUsername")
elem.send_keys(username)
elem.send_keys(Keys.RETURN)

elem = driver.find_element_by_name("tbxPassword")
elem.send_keys(password)
elem.send_keys(Keys.RETURN)

In [ ]:
result = None
for study_id in tqdm(study_ids):
    elem = driver.find_element_by_name("tbxAssignedID")
    elem.clear()
    elem.send_keys(study_id)
    elem.send_keys(Keys.RETURN)

    try:
        soup = BeautifulSoup(driver.page_source, "lxml")
        table = soup.find_all("table")[3]

        df = pd.read_html(str(table), parse_dates=True, header=0)[0]
        df["Study DateTime"] = pd.to_datetime(
            df["Study DateTime"], infer_datetime_format=True)
        df.drop(["Unnamed: 0"], inplace=True, axis=1)
    except KeyError:
        d = {"Study DateTime": [np.nan], "StudyDescription": [
            np.nan], "Modality": [np.nan]}
        df = pd.DataFrame(data=d)
    df["study_id"] = study_id

    if result is None:
        result = df
    else:
        result = pd.concat([df, result])

In [8]:
result.to_csv("imaging_metadata.csv", index=False)